In [1]:
# Import Dependancies
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from config import conn_str

In [2]:

engine = create_engine(conn_str)
conn = engine.connect()

In [3]:
# Create variable for Salaries Table Information
salaries_table = pd.read_sql("SELECT * FROM salaries", conn)
salaries_table.head()

,emp_no,salary,from_date,to_date
0,10001,60117,1986-06-26,1987-06-26
1,10002,65828,1996-08-03,1997-08-03
2,10003,40006,1995-12-03,1996-12-02
3,10004,40054,1986-12-01,1987-12-01
4,10005,78228,1989-09-12,1990-09-12


In [4]:
# Create variable for Titles Table Information
titles_table = pd.read_sql("SELECT * FROM titles", conn)
titles_table.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


In [5]:
# Combine Data from "Salaries" & "Titles" Tables
combined_tables = pd.merge(titles_table, salaries_table, on="emp_no", how="inner", suffixes=( '_(Titles)', '_(Salaries)'))
combined_tables.head()

,emp_no,title,from_date_(Titles),to_date_(Titles),salary,from_date_(Salaries),to_date_(Salaries)
0,10001,Senior Engineer,1986-06-26,9999-01-01,60117,1986-06-26,1987-06-26
1,10002,Staff,1996-08-03,9999-01-01,65828,1996-08-03,1997-08-03
2,10003,Senior Engineer,1995-12-03,9999-01-01,40006,1995-12-03,1996-12-02
3,10004,Engineer,1986-12-01,1995-12-01,40054,1986-12-01,1987-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01,40054,1986-12-01,1987-12-01


In [6]:
# Pare down by Title and Salary
pared_down_df = combined_tables[['title', 'salary']]
pared_down_df.head()

,title,salary
0,Senior Engineer,60117
1,Staff,65828
2,Senior Engineer,40006
3,Engineer,40054
4,Senior Engineer,40054


In [11]:
# Convert salary column to a number, it was erroring out becasue it was a non-numeric data type and 
# returning the error "DataError: No numeric types to aggregate"

pared_down_df.loc[:,'salary'] = pd.to_numeric(pared_down_df.loc[:,'salary'])

In [14]:
grouped_df = round(pared_down_df.groupby('title').mean())
grouped_df


,salary
title,
Assistant Engineer,48493.0
Engineer,48540.0
Manager,51531.0
Senior Engineer,48507.0
Senior Staff,58503.0
Staff,58465.0
Technique Leader,48581.0


In [13]:
# Set x_axis, y_axis & Tick Locations
x_axis = grouped_df['title']
ticks = np.arange(len(x_axis))
y_axis = grouped_df['salary']
 
# Create Bar Chart Based on Above Data
plt.bar(x_axis, y_axis, align="center", alpha=0.5, color=["k", "r", "g", "m", "b", "c", "y"])

# Create Ticks for Bar Chart's x_axis
plt.xticks(ticks, x_axis, rotation="vertical")

# Set Labels & Title
plt.ylabel("Salaries ($)")
plt.xlabel("Employee Titles")
plt.title("Average Employee Salary by Title")

# Save Figure
plt.savefig("./Images/average_salary_by_title.png")

# Show plot
plt.show()

KeyError: 'title'